# COVID-19 Data  in France

Data available from the goverment sources at https://www.data.gouv.fr

General view:

Offers a dataset with a summary of all data collected, available at

https://www.data.gouv.fr/en/datasets/donnees-relatives-a-lepidemie-de-covid-19-en-france-vue-densemble/

and the permanent link for data download is:

https://www.data.gouv.fr/en/datasets/r/d3a98a30-893f-47f7-96c5-2f4bcaaa0d71

Hospitalization data:
Provide a summary for hospital's state, available at:

https://www.data.gouv.fr/en/datasets/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/

In [157]:
import pandas as pd
import numpy as np
import matplotlib
import plotly
import datetime

# import local functions
import covid19_analysis.dataFun as df
import covid19_analysis.dataPlot as dplot

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Define path from online repo
repo_url='https://www.data.gouv.fr/en/datasets/r/d3a98a30-893f-47f7-96c5-2f4bcaaa0d71'

# Get data from online repo
df_fr=pd.read_csv(repo_url, parse_dates=['date'])

## General view of the crisis evolution

In [67]:
dplot.disp_cumulative(df_fr, 'France', source='datagouv')

## Growing rate over time

In [86]:
# Check growth ratio
cases_ts = pd.Series(data=df_fr.total_cas_confirmes.values, index=df_fr.date, dtype = int)

start_date = '2020-07-01'
mask = cases_ts.index > start_date
fig = dplot.growth_rates(cases_ts[mask], trend_line=True)

In [87]:
# Check fatalities growth ratio
fatalities_ts = pd.Series(data=df_fr.total_deces_hopital.values, index=df_fr.date, dtype = int)

mask = cases_ts.index > start_date
fig = dplot.growth_rates(fatalities_ts[fatalities_ts.index >= start_date],label='Fatalities ', \
                         trend_line=True, y_range=[1, 1.01])

## Check doubling rates of cases

In [88]:
# compare to general doubliting ratios
dplot.growing_ratio_country(df_fr, 100, df_source='datagouv', date_filter='2020-08-07', clear_pop=True)

From the first week of August the trend Cases and Fatalies is growing. fatatilies mostly follows a weakly doubling behaviour. The cases trend has change its behaviour for the last days and follows a trend of 5 days doubling trend.


## Activate cases

In [90]:
dplot.disp_current_cases(df_fr, 'France', pop_factor=1000, source='datagouv')

From August 2020, the number of confirmed cases has been increasing. However the number of fatalies has not follow the same behaviour.
The evolution of the disease in France has a steady growth (21 March 2020) for confirmed cases and a almost exponencial grow for fatalities. The next graph display the daily evolution of the virus

## Daily cases progression

In [121]:
dplot.disp_daily_cases(df_fr, 'France', 'datagouv')

# Data from hospitalization in france

In [134]:
# Nombre cumulé de services hospitaliers ayant déclaré au moins un cas
#data_hospital_url = 'https://www.data.gouv.fr/en/datasets/r/41b9bd2a-b5b6-4271-8878-e45a8902ef00'

# donnees-hospitalieres-classe-age-covid19-2020
data_hospital_url = 'https://www.data.gouv.fr/en/datasets/r/08c18e08-6780-452d-9b8c-ae244ad529b3'
df_hospitals = pd.read_csv(donnees_hospitalieres_etablissements, sep=';')

## Situation in Auvergne-Rhône-Alpes region

In [292]:
df_reg = df_hospitals[df_hospitals.reg==84] # Auvergne-Rhône-Alps

In [293]:
# display current cases in hospital divided by age
fig = plotly.graph_objs.Figure()

for age_range in df_reg.cl_age90.unique():
    df_ar = df_reg[df_reg.cl_age90==age_range]  
    
    if age_range == 0:
        label_name = 'all'
    else:
        label_name = str(age_range)
        
    fig.add_trace(
        plotly.graph_objs.Scatter(
            mode = 'lines',
            name = label_name,
            x = df_ar.jour,
            y = df_ar.hosp,
            line=dict(width = 1.5),
        )
    )

fig.update_layout(
    plot_bgcolor='white', 
    xaxis_title = 'Dates [Days]',
    yaxis_title = 'Cases',
    title = 'Hospitalization cases evolution' + datetime.datetime.today().strftime(', %B %d, %Y'),
    title_x = .5
)

## Density of cases based on age range and time

Explore the quantity of existing cases in the region according to the range of age

In [295]:
df_reg2 = df_reg[df_reg.cl_age90 != 0] # exclude all cases

In [291]:
# group dataset by days and age-range
piv = pd.pivot_table(df_reg2, values='hosp',index=['cl_age90'], columns=['jour'], fill_value=0)

fig = plotly.graph_objs.Figure()

fig.add_trace(
    plotly.graph_objs.Heatmap(
        z = piv,
        x = piv.columns,
        y = piv.index,
    )
) 

fig.update_layout(
    title='Age range heatmap',
    #xaxis_nticks=36)
)

fig.update_layout(
    plot_bgcolor='white', 
    xaxis_title = 'Dates [Days]',
    yaxis_title = 'Age range',
    title = 'Corona cases according to the age' + datetime.datetime.today().strftime(', %B %d, %Y'),
    title_x = .5
)
fig.show()

Select region and data source from dataset

In [99]:
reg_std = 'Auvergne-Rhône-Alpes'
# sources: 'Santé publique France', 'Ministère des Solidarités et de la Santé', 'ARS Auvergne-Rhône-Alpes', 'OpenCOVID19-fr'
src_name = 'OpenCOVID19-fr'
df_reg = df_c_raw[(df_c_raw.maille_nom == reg_std) & (df_c_raw.source_nom == src_name)] #

Display the evolution of the coronavirus in Auvergne-Rhône-Alpes for the last months, for the dataset with source from Sante publique France, doest not display the number of fatalities, compare to the data source ARS AuvergneRhoneAlpes. for the moment the chart use the ARS source (21 March 2020)

In [100]:
dplot.disp_cumulative(df_reg, reg_std)

Data for Auverne Rhône Alpes is not updated, only fatalities are reported

In [101]:
dplot.disp_daily_cases(df_reg, reg_std, 'SPF')

In [103]:
# list of all deparments availables
print('List of all departments availables\n', df_c_raw[df_c_raw.granularite == 'departement'].maille_nom.unique())

List of all departments availables
 ['Charente' 'Charente-Maritime' 'Corrèze' 'Creuse' 'Dordogne' 'Gironde'
 'Landes' 'Lot-et-Garonne' 'Pyrénées-Atlantiques' 'Deux-Sèvres' 'Vienne'
 'Haute-Vienne' 'Hérault' 'Haute-Savoie' 'Aisne' 'Doubs' 'Nord' 'Oise'
 'Pas-de-Calais' 'Somme' 'Territoire de Belfort' "Côte-d'Or" 'Finistère'
 'Loire-Atlantique' 'Bas-Rhin' 'Alpes-Maritimes' 'Maine-et-Loire'
 'Mayenne' 'Seine-Maritime' 'Ille-et-Vilaine' 'Morbihan' 'Sarthe' 'Ain'
 'Ardennes' 'Aube' 'Eure' 'Marne' 'Haute-Marne' 'Meurthe-et-Moselle'
 'Meuse' 'Moselle' 'Haut-Rhin' 'Rhône' 'Vosges' 'Allier' 'Ardèche'
 'Cantal' 'Drôme' 'Gard' 'Isère' 'Loire' 'Haute-Loire' 'Puy-de-Dôme'
 'Saône-et-Loire' 'Savoie' 'Aveyron' 'Bouches-du-Rhône' "Côtes-d'Armor"
 'Eure-et-Loir' 'Indre-et-Loire' 'Haute-Saône' 'Vaucluse' 'Guyane'
 'Hautes-Alpes' 'Calvados' 'Cher' 'Corse-du-Sud' 'Haute-Corse'
 'Haute-Garonne' 'Indre' 'Loir-et-Cher' 'Loiret' 'Manche' 'Paris'
 'Seine-et-Marne' 'Yvelines' 'Var' 'Essonne' 'Hauts-de-Seine'
 '

## Situation in Savoie department

In [107]:
dep_std = 'Savoie'
dep_src = 'ARS Auvergne-Rhône-Alpes'
df_dep = df_c_raw[(df_c_raw.maille_nom == dep_std)] # & (df_c_raw.source_nom == dep_src)

In [108]:
df_c_raw[(df_c_raw.maille_nom == dep_std)].source_nom.unique()

array(['ARS Auvergne-Rhône-Alpes', 'Santé publique France Data'],
      dtype=object)

In [111]:
dplot.disp_cumulative(df_dep,dep_std)

In [112]:
dplot.disp_daily_cases(df_dep, dep_std, 'SPF')

They only update the number of fatalities for savoie department!

In [116]:
df_c_raw[df_c_raw.granularite == 'region'].maille_nom.unique()

array(['Île-de-France', 'Nouvelle-Aquitaine', 'Auvergne-Rhône-Alpes',
       'Bourgogne-Franche-Comté', 'Hauts-de-France', 'Grand Est',
       'Guadeloupe', 'Martinique', 'Guyane', 'La Réunion', 'Mayotte',
       'Centre-Val de Loire', 'Normandie', 'Pays de la Loire', 'Bretagne',
       'Occitanie', "Provence-Alpes-Côte d'Azur", 'Corse'], dtype=object)

## Situation in Ile de France

In [117]:
reg_std = 'Île-de-France'
# sources: 'Santé publique France', 'Ministère des Solidarités et de la Santé', 'ARS Auvergne-Rhône-Alpes', 'OpenCOVID19-fr'
src_name = 'OpenCOVID19-fr'
df_reg = df_c_raw[(df_c_raw.maille_nom == reg_std) ] # & (df_c_raw.source_nom == src_name)

In [119]:
df_reg

,date,granularite,maille_code,maille_nom,cas_confirmes,cas_ehpad,cas_confirmes_ehpad,cas_possibles_ehpad,deces,deces_ehpad,reanimation,hospitalises,nouvelles_hospitalisations,nouvelles_reanimations,gueris,depistes,source_nom,source_url,source_archive,source_type
15,2020-01-24,region,REG-11,Île-de-France,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,Santé publique France,https://www.santepubliquefrance.fr/maladies-et...,https://web.archive.org/web/20200124202351/htt...,sante-publique-france
16,2020-01-24,region,REG-11,Île-de-France,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,Ministère des Solidarités et de la Santé,https://solidarites-sante.gouv.fr/IMG/pdf/2001...,https://web.archive.org/web/20200322183046/htt...,ministere-sante
23,2020-01-25,region,REG-11,Île-de-France,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,Santé publique France,https://www.santepubliquefrance.fr/maladies-et...,https://web.archive.org/web/20200126024644/htt...,sante-publique-france
28,2020-01-26,region,REG-11,Île-de-France,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,Santé publique France,https://www.santepubliquefrance.fr/maladies-et...,https://web.archive.org/web/20200127144431/htt...,sante-publique-france
35,2020-01-27,region,REG-11,Île-de-France,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,Santé publique France,https://www.santepubliquefrance.fr/maladies-et...,https://web.archive.org/web/20200128153608/htt...,sante-publique-france
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29786,2020-10-21,region,REG-11,Île-de-France,NaN,NaN,NaN,NaN,8549.0,NaN,672.0,3515.0,427.0,70.0,37906.0,NaN,OpenCOVID19-fr,NaN,NaN,opencovid19-fr
29907,2020-10-22,region,REG-11,Île-de-France,NaN,NaN,NaN,NaN,8598.0,NaN,703.0,3685.0,484.0,84.0,38143.0,NaN,OpenCOVID19-fr,NaN,NaN,opencovid19-fr
30028,2020-10-23,region,REG-11,Île-de-France,NaN,NaN,NaN,NaN,8644.0,NaN,730.0,3908.0,542.0,85.0,38404.0,NaN,OpenCOVID19-fr,NaN,NaN,opencovid19-fr
30149,2020-10-24,region,REG-11,Île-de-France,NaN,NaN,NaN,NaN,8685.0,NaN,734.0,3971.0,341.0,63.0,38637.0,NaN,OpenCOVID19-fr,NaN,NaN,opencovid19-fr


In [118]:
dplot.disp_cumulative(df_reg, reg_std)

Data for Auverne Rhône Alpes is not updated, only fatalities are reported

In [101]:
dplot.disp_daily_cases(df_reg, reg_std, 'SPF')